In [28]:
from locnavig import *
from IPython.display import Audio

In [3]:
try:
    client = ClientAsync()
    node = await client.wait_for_node()
    await node.lock()
    print("Thymio connected")
except:
    print("Thymio not connected")

Thymio connected


In [6]:
spped_gain= 2
motor_speed = 100   # speed of the robot 
LTobst = 5      # low obstacle threshold to switch state 1->0
HTobst = 13    # high obstacle threshold to switch state 0->1
obst_gain = 15  # /100 (actual gain: 15/100=0.15)

state = 0      # Actual state of the robot: 0->global navigation, 1->obstacle avoidance
case = 0       # Actual case of obstacle avoidance: 0-> left obstacle, 1-> right obstacle, 2-> obstacle in front  
side = bool(random.randint(0,1)) # we randomly generate the first bypass choice of the robot when he encounters an obstacle in front of him  

rotation_time = 1  # 1ms time before rotation 
step_back_time = 1 # 1ms time during which i step back 

prox_horizontal = [0, 0, 0, 0, 0] #initialize the value of the proximal sensors to 0

In [7]:
async def prog():
    while True:
        await node.wait_for_variables({"prox.horizontal"})
        for i in range(0, 5):
            prox_horizontal[i]=list(node.v.prox.horizontal)[i]  
        await client.sleep(0.01)
        obst = [prox_horizontal[0], prox_horizontal[1], prox_horizontal[2], prox_horizontal[3], prox_horizontal[4]]

        await avoid_obstacles(node,client,state,side,obst,HTobst, LTobst, motor_speed, step_back_time,spped_gain, rotation_time)
        
await prog()

CancelledError: 

In [8]:
await node.set_variables(set_motor_speed(0, 0))  

In [12]:
leds = [0, 0, 0, 0, 0, 0, 0, 0]
rotation_speed =  1000
rotation_time = 1
rainbow = [
        (255, 255, 0),  # Jaune
        (112, 12, 0), #Orange
        (255, 0, 0),    # Rouge
        (205, 20, 64),  #Rose
        (102, 0, 153), #Mauve
        (0, 0, 255),    # Bleu
        (0, 255, 127),  # Cyan
        (0, 255, 0),    # Vert
        ]

In [17]:
async def we_did_it(node, client,rotation_speed,rotation_time,rainbow):
    #await node.set_variables(set_motor_speed(400, -400))
    for colors in rainbow:            # Mettre à jour les LEDs avec la nouvelle couleur
        await node.set_variables(set_leds(colors,colors,colors))
        await rotate_90_degrees_clockwise(node,client,rotation_speed, rotation_time)
        await step_back(node,client,rotation_speed, rotation_time)
        await rotate_90_degrees_counterclockwise(node,client,rotation_speed, rotation_time)
        await move_forward(node,client,rotation_speed, rotation_time)
        await client.sleep(1)
    await node.set_variables(set_motor_speed(0, 0))  

In [19]:
await we_did_it(node, client,rotation_speed,rotation_time,rainbow)

In [15]:
async def we_did_it(rotation_speed,rotation_time,rainbow):
    for colors in rainbow:            # Mettre à jour les LEDs avec la nouvelle couleur
        #circle = [colors, colors, colors, colors,colors,colors,colors,colors]
        await node.set_variables(set_leds(colors,colors,colors))
        await rotate_90_degrees_clockwise(rotation_speed, rotation_time+2)
    await node.set_variables(set_motor_speed(0, 0))  

In [ ]:
async def update_speed(state, motor_speedl, motor_speedr):
    if not state:
        await node.set_variables(set_motor_speed(motor_speedl, motor_speedr)) 
    else:
        pass        